<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Resnet-transfer-learning-script-note-book." data-toc-modified-id="Resnet-transfer-learning-script-note-book.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Resnet transfer learning script note book.</a></span><ul class="toc-item"><li><span><a href="#Import-modules-into-notebook-for-analysis" data-toc-modified-id="Import-modules-into-notebook-for-analysis-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import modules into notebook for analysis</a></span></li><li><span><a href="#Resnet-parameterisation" data-toc-modified-id="Resnet-parameterisation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Resnet parameterisation</a></span></li><li><span><a href="#Resnet-Setup" data-toc-modified-id="Resnet-Setup-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Resnet Setup</a></span></li></ul></li></ul></div>

# Resnet transfer learning script note book. 

The purpose of this module is to complete training via tranfer learning using Resnet50 module to analyze 

## Import modules into notebook for analysis

-The purpose of this section is to import appropriate modules from tensorflow,numpy, pandas etc prior to performing training Resnet with image dataset 

-Please note that numpy module version >1.14.5 is required in order for tensorflow import to work. 



In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2

import os

In [ ]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import Sequential


In [18]:
from tensorflow.python.keras import optimizers
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [28]:
os.getcwd()


'\\\\nsq024vs\\u8\\aczd087\\MyDocs\\Data science masters\\Computer vision\\Coursework'

In [41]:
#Specialist folders for dataset splitting of folders into train and test. 

import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
input_folder='/Dataset/resizedimages_colour_grp_227_227'
output_folder='/Dataset/train_val_test_sets_227_227'

split_folders.ratio(r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Computer vision\Coursework\Dataset\resizedimages_colour_grp_227_227', 
                    output=r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Computer vision\Coursework\Dataset\train_val_test_sets_227_227', 
                    seed=1337, 
                    ratio=(.7, .3, 0)) # default values


## Resnet parameterisation

-The purpose of this section is to parameterise Resnet for analysis

In [55]:
# Fixed for our Cats & Dogs classes
NUM_CLASSES = 53

No of color channels present
CHANNELS = 3

#Resizing image to fit into model 
IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_TESTING= 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_TESTING= 100

#Parameteristion of optimisers
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

## Resnet Setup 

-- The purpose of this step is to remove Resnet models final layer structure and import weights from 

In [10]:
#Change directory to location of Resnet dataset

os.chdir(r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Computer vision\Coursework')

In [11]:
#Change directory of Resnet 

resnet_weights_path ='Dataset\Resnet weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [14]:
#Still not talking about our train/test data or any pre-processing.

model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False


In [19]:
image_size=IMAGE_RESIZE

In [44]:
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
#Creation of training dataset

train_generator = data_generator.flow_from_directory(
        r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Computer vision\Coursework\Dataset\train_val_test_sets_227_227\train',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

In [ ]:
#Creation of validation dataset 

testing_generator = data_generator.flow_from_directory(
        r'\\nsq024vs\u8\aczd087\MyDocs\Data science masters\Computer vision\Coursework\Dataset\train_val_test_sets_227_227\test',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical') 

In [59]:
# Early stopping & checkpointing the best model in ../working dir & restoring that as our model for prediction
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = 'weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                                  monitor = 'val_loss', 
                                  save_best_only = True, 
                                  mode = 'auto')

In [ ]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=testing_generator,
        validation_steps=STEPS_PER_EPOCH_TESTING,
        callbacks=[cb_checkpointer, cb_early_stopper]
)
#odel.load_weights("../working/best.hdf5")

Epoch 1/10
29/29 [==============================] - 2667s 92s/step - loss: 1.8255 - acc: 0.6800 - val_loss: 2.9407 - val_acc: 0.2802
Epoch 2/10
29/29 [==============================] - 2634s 91s/step - loss: 1.1174 - acc: 0.8324 - val_loss: 2.4308 - val_acc: 0.4027
Epoch 3/10
18/29 [=================>............] - ETA: 14:21 - loss: 0.8513 - acc: 0.8796

In [ ]:
print(fit_history.history.keys())